# Homework Assignment 2: Data Prep
In this homework assignment, you will continue your exploration of the [SWAN-SF Dataset](https://doi.org/10.7910/DVN/EBCFKM), described in the paper found [here](https://doi.org/10.1038/s41597-020-0548-x).


This assignment will have you explore the cardinalities, number of missing values, detect outliers, handle missing values and outliers, and create data quality report for original and cleaned dataset.

## Step 1: Downloading the Data

This assignment will only be using [Partition 1](https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/EBCFKM/BMXYCB). Recall that in Homework 1, we started to construct the analytics base table for our [SWAN-SF Dataset](https://doi.org/10.7910/DVN/EBCFKM). In that assignment, we read the data from the two subdirectories, __FL__ and __NF__, of the __partition1__ direcotry. These two subdirectories represented the two classes of our target feature in the solar flare prediction problem we will be attempting to solve this semester. We then processed these samples of multivariate time series to construct descriptive features for each sample, and then placed them into our analytics base table.

In this assignment, you will be utilizing a set of extracted descriptive features much like what you were asked to construct in Homework 1. However, this dataset contains many more extracted features than you were asked to compute for Homework 1 (>800), so we need to explore the data to find data quality issues and identify ways to address these issues. Below are links to the full extracted feature dataset for partition 1 and a toy dataset to use for testing you functions.

- [Full Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/partition1ExtractedFeatures.csv)
- [Toy Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/toy_partition1ExtractedFeatures.csv)

Now that you have the extracted features csv files, you will load that data into a Pandas DataFrame using the [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) method.  


In [ ]:
import os
import pandas as pd
from pandas import DataFrame 
import numpy as np

### Read the dataset

In [ ]:
data_dir = '/data/MVTS'
data_file = 'toy_partition1ExtractedFeatures.csv'

In [ ]:
abt = pd.read_csv(os.path.join(data_dir, data_file), index_col=0)

### Q1 (25 points)

Write a function to extract the various pieces of a data quality report, for a specific attribute, and return a dataframe with this information.

 * 'Feature Name': Contains the time series statistical feature name
 
 * 'Cardinality': Contains the count of unique values for the feature
            
 * 'Non-null Count': Contains the number of non-null entries for the feature
            
 * 'Null Count': Contains the number of null or missing entries for the feature
            
 * 'Min': Contains the minimum value of the feature (Without considering the null or nan value)
 
 * '25th': Contains the first quartile (25%) of the feature values (Without considering the null/nan value)
 
 * 'Mean': Contains the mean of the feature values (Without considering the null/nan value)
 
 * '50th': Contains the median of the feature values (Without considering the null/nan value)
            
 * '75th': Contains the third quartile (75%) of the feature values (Without considering the null/nan value)
 
 * 'Max': Contains the maximum value of the feature (Without considering the null/nan value),
            
 * 'Std. Dev': Contains the standard deviation of the feature (Without considering the null/nan value)
 
In addition to the values above, you should identify the number of upper and lower outliers using the $val < Q1-1.5IQR$ and $val > Q3+1.5IQR$ outlier identification method. These added features should be called `Outlier Count Low` and `Outliers Count High` respectively.


 
 Some useful functions for this can be found at:
 
 * [Numpy.percentile](https://numpy.org/doc/stable/reference/generated/numpy.percentile.html)
 
 * [pandas.isna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.isna.html)
 
 * [Numpy.mean](https://numpy.org/doc/stable/reference/generated/numpy.mean.html)
 
 * [Numpy.std](https://numpy.org/doc/stable/reference/generated/numpy.std.html)
 

In [ ]:
def calc_summary_for(feature_name:str, data:DataFrame) -> DataFrame:
    summary_feature_names = ['Feature Name', 'Cardinality', 'Non-null Count', 'Null Count', 'Min', '25th', 'Mean', 
                             '50th', '75th', 'Max', 'Outlier Count Low', 'Outlier Count High', 'STD']
    
    iqr = (data[feature_name].quantile(q=.75) - data[feature_name].quantile(q=.25)) * 1.5
    lower = 0
    upper = 0
    for y in data[feature_name].values:
        if y < (data[feature_name].quantile(q=.25) - iqr):
            lower +=1
        elif y> (iqr + data[feature_name].quantile(q=.25)):
            upper +=1
        else:
            continue

    temp_list = [feature_name, data[feature_name].nunique(), data[feature_name].notnull().sum(), data[feature_name].isnull().sum(), data[feature_name].min(), data[feature_name].quantile(q=.25),
    data[feature_name].mean(), data[feature_name].quantile(q=.50), data[feature_name].quantile(q=.75), data[feature_name].max(), lower, upper, data[feature_name].std()]

    df = pd.DataFrame(data={k:[v] for k,v in zip(summary_feature_names, temp_list)}) # dataframe
    df.set_index(['Feature Name'], inplace=True)
    return df

In [ ]:
summary_TOTUSJH_min = calc_summary_for('TOTUSJH_min', abt)

### Q2 (25 points)
Using what you produced to answere Q1, you should now write a function to construct the data quality report for all of the numerical features of our dataset.  You should loop over all of the features in the analytics base table represented by the input feature dataset files from partition 1, with the exception of the first column (this is the index column if you read the file correctly), and the `id`, `lab`, `st`, and `et` columns.  

Your output from this function will be a DataFrame that has 1 row for each feature. 

In [ ]:
def construct_quality_report(data:DataFrame) -> DataFrame:
    excluded_columns = ['id', 'lab', 'st', 'et']
    
    summary_feature_names = ['Feature Name', 'Cardinality', 'Non-null Count', 'Null Count', 'Min', '25th', 'Mean', 
                             '50th', '75th', 'Max', 'Outlier Count Low', 'Outlier Count High', 'STD']
                     
    data.drop(columns=excluded_columns, inplace=True)
    analyzed = [calc_summary_for(feature_name, data) for feature_name, values in data.iteritems()]

    return pd.concat(analyzed) # dataframe of dataframes

In [ ]:
summary_table = construct_quality_report(abt)

### Q3 (25 points)
#### Drop features with improper cardinality:
Using the quality report summary table that is returned from the function you wrote for Q2, we are now going to investigate our data. For this, you should use the table returned for the [Full Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/partition1ExtractedFeatures.csv) and not the toy dataset I provided for testing.

Since we are using real valued features, a majority of them shall have a cardinality close to the sample count. So, for this question, you are to write a function that takes in the summary table and the input dataset DataFrame, and drops the feature that have a cardinality less than 10. This feature should be dropped from both the data quality report summary table and from the actual input dataset DataFrame.

A useful method for this operation is:

* [pandas.DataFrame.drop](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) (Make sure to use the inplace option otherwise it returns a copy)

In [ ]:
def drop_low_card_data(summary_table:DataFrame, data:DataFrame) -> None:
    summary_table.drop(summary_table.loc[summary_table['Cardinality'] < 10].index.tolist(), inplace=True)
    data.drop(columns=summary_table.loc[summary_table['Cardinality'] < 10].index.tolist(), axis=1, inplace=True)

In [ ]:
drop_low_card_data(summary_table, abt)

### Q4 (25 points)
#### Drop features with excessive NaN

Again, using the quality report summary table that is returned from the function you wrote for Q2, we are going to continue investigating our data. For this, you should still be using the table returned for the [Full Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/partition1ExtractedFeatures.csv) and not the toy dataset I provided for testing.

Like the features that were dropped for Q3, some of the extracted features don't work on all of the variates of the input multi-variate time series samples very well.  So, some of these features return an excessive number of not a number values.  These are not verry useful features, so we want to get rid of them before we continue. To do this, you are to write a function that takes in the summary table and the input dataset DataFrame, and drops the features that have more than 1% of the entries as null/nan values. Again, these features should be dropped from both the data quality report summary table and from the actual input dataset DataFrame.

As in Q3, a useful method for this operation is:

* [pandas.DataFrame.drop](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) (Make sure to use the inplace option otherwise it returns a copy)

In [ ]:
def drop_excessive_nan_data(summary_table:DataFrame, data:DataFrame) -> None:
    summary_table['Null Count'] = ["" if ((i * 100) / len(summary_table['Null Count'].to_numpy())) > .01 else i 
    for i in summary_table['Null Count'].to_numpy()]
    summary_table.drop(summary_table.loc[summary_table['Null Count'] == ""].index.tolist(), inplace=True)
    data.drop(columns=summary_table.loc[summary_table['Null Count'] == ""].index.tolist(), axis=1, inplace=True)

In [ ]:
drop_excessive_nan_data(summary_table, abt)

Now that you have the data cleaned up a little, save the results of both your summary table and your analytics base table using the [pandas.to_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html) method. We will want to use these results for the next homework assignment.

In [ ]:
out_dir = "your output directory"
out_summary_table_name = 'data_quality_table.csv'
out_abt_name = 'cleaned_partition1ExtractedFeatures.csv'

In [ ]:
summary_table.to_csv(os.path.join(out_dir, out_summary_table_name))
abt.to_csv(os.path.join(out_dir, out_abt_name))